In [1]:
import pandas as pd
import numpy as np


In [2]:
#reading dataset
df = pd.read_csv('train.csv',encoding= 'unicode_escape')  
#val_df = pd.read_excel('val.xlsx')
test_df = pd.read_csv('val.csv',encoding= 'unicode_escape')


In [3]:
train_df=df.sample(frac = 0.4)

print('size of training set: %s' % (len(train_df['Utterance'])))
print('size of validation set: %s' % (len(test_df ['Utterance'])))


train_df.head(10)

size of training set: 6040
size of validation set: 3100


,Utterance,Intent
10045,i can't remember when my car last went to the ...,last_maintenance
2656,can you confirm my reservation for 6 pm on the...,confirm_reservation
5956,please give me today's date,date
838,would you be able to tell me how to make a vac...,pto_request
8320,can i book a meeting room from 2:00 pm to 3:00 pm,schedule_meeting
1781,toss a coin i will take tails,flip_coin
3468,help me get access to my recent transaction hi...,transactions
3011,it takes how long to get to the beach in santa...,distance
8197,will you tell me my routing number,routing
11797,can you help me spell facetiousness,spelling


In [4]:
train_df.info()
print()
test_df.info()
print()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6040 entries, 10045 to 9811
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Utterance  6040 non-null   object
 1   Intent     6040 non-null   object
dtypes: object(2)
memory usage: 141.6+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3100 entries, 0 to 3099
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Utterance  3100 non-null   object
 1   Intent     3100 non-null   object
dtypes: object(2)
memory usage: 48.6+ KB



In [5]:
train_df.head()

,Utterance,Intent
10045,i can't remember when my car last went to the ...,last_maintenance
2656,can you confirm my reservation for 6 pm on the...,confirm_reservation
5956,please give me today's date,date
838,would you be able to tell me how to make a vac...,pto_request
8320,can i book a meeting room from 2:00 pm to 3:00 pm,schedule_meeting


In [6]:
test_df.head()

,Utterance,Intent
0,"in spanish, meet me tomorrow is said how",translate
1,"in french, how do i say, see you later",translate
2,how do you say hello in japanese,translate
3,how do i ask about the weather in chinese,translate
4,"how can i say ""cancel my order"" in french",translate


In [7]:
train_df["Intent"].value_counts()

how_busy             54
travel_alert         51
smart_home           50
plug_type            50
pto_used             49
                     ..
spelling             31
what_is_your_name    31
jump_start           31
nutrition_info       28
traffic              28
Name: Intent, Length: 151, dtype: int64

In [8]:
test_df["Intent"].value_counts()

oos                   100
pin_change             20
account_blocked        20
what_song              20
international_fees     20
                     ... 
change_speed           20
tire_pressure          20
no                     20
apr                    20
change_accent          20
Name: Intent, Length: 151, dtype: int64

In [9]:
Train_review = train_df["Utterance"].values.tolist()
Train_sentiment = train_df["Intent"].tolist() 

In [10]:
Test_review = test_df["Utterance"].values.tolist()
Test_sentiment = test_df["Intent"].tolist() 

In [11]:
x_train, x_test, y_train, y_test = Train_review, Test_review, Train_sentiment, Test_sentiment

In [12]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500)


In [13]:
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 8
	95percentile : 14
	99percentile : 17


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 8
	95percentile : 15
	99percentile : 19


In [14]:
model = t.get_classifier()

In [15]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [16]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
1007/1007 [==============================] - 10148s 10s/step - loss: 4.8410 - accuracy: 0.0364 - val_loss: 4.3670 - val_accuracy: 0.1810
Epoch 2/4
1007/1007 [==============================] - 10002s 10s/step - loss: 3.5935 - accuracy: 0.5124 - val_loss: 2.7359 - val_accuracy: 0.7197
Epoch 3/4
1007/1007 [==============================] - 10559s 10s/step - loss: 2.0196 - accuracy: 0.8485 - val_loss: 1.5767 - val_accuracy: 0.8555
Epoch 4/4
1007/1007 [==============================] - 10184s 10s/step - loss: 1.2518 - accuracy: 0.9515 - val_loss: 1.2755 - val_accuracy: 0.8865


In [17]:
learner.validate(val_data=(val),)

              precision    recall  f1-score   support

           0       0.90      0.95      0.93        20
           1       0.86      0.95      0.90        20
           2       1.00      1.00      1.00        20
           3       0.87      1.00      0.93        20
           4       1.00      0.95      0.97        20
           5       1.00      0.95      0.97        20
           6       0.89      0.80      0.84        20
           7       0.74      0.85      0.79        20
           8       0.83      1.00      0.91        20
           9       1.00      1.00      1.00        20
          10       1.00      0.90      0.95        20
          11       0.83      1.00      0.91        20
          12       0.75      0.75      0.75        20
          13       0.94      0.80      0.86        20
          14       0.87      1.00      0.93        20
          15       1.00      0.90      0.95        20
          16       0.83      0.95      0.88        20
          17       0.95    

array([[19,  0,  0, ...,  0,  0,  0],
       [ 0, 19,  0, ...,  0,  0,  0],
       [ 0,  0, 20, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 20,  0,  0],
       [ 0,  0,  0, ...,  0, 19,  0],
       [ 0,  0,  0, ...,  0,  0, 19]], dtype=int64)